In [ ]:
#!pip install konlpy

In [ ]:
import os
import pandas as pd
import re
from konlpy.tag import Okt
from tqdm import tqdm

# Okt 로드
okt = Okt()

# 불용어 리스트
with open("stopwords-ko.txt", "r", encoding="utf-8") as f:
    stopwords = set([line.strip() for line in f if line.strip()])

# 복합어(신조어 등) 치환 사전
compound_words = [
    '가성비', '글라시에', '보냉', '세일', '잘', '색깔별로', '피오니', '사이즈', '벤티', '오리지널', '수밀감', '수준급', '충동구매',
    '충동 구매', '신학기', '그립감', '선물용', '허접', '사은품', '빈플러스', '이중구조', '재구매', '데이오프', '킨토', '연마제', '오랜만',
    '속상', '샌드 베이지', '보온', '끄떡없', '강추', '추운날', '보온력', '샌드베이지'
]

# 복합어 치환 함수
def merge_compounds(text):
    for word in compound_words:
        if word in text:
            # 복합어를 _로 이어서 치환 (예: '가성비' -> '가_성_비')
            text = text.replace(word, '_'.join(list(word)))
    return text

# 전처리 함수 정의
def preprocess_review(review):
    if pd.isna(review):
        return []

    # 복합어 치환
    review = merge_compounds(review)

    # 특수문자 제거 및 정리
    review = re.sub(r'[^\uAC00-\uD7A3a-zA-Z0-9\s]', '', str(review)).strip()
    review = re.sub(r'[^\uAC00-\uD7A3 0-9\s]', '', review).strip()

    # 형태소 분석
    morphs = okt.pos(review, stem=True)

    tokens = []
    for word, tag in morphs:
        if word in stopwords:
            continue
        if tag in ['Verb', 'Adjective']:
            tokens.append(word)
        elif tag in ['Noun', 'Alpha']:
            tokens.append(word)
        elif tag == 'Modifier':
            tokens.append(word)
    return tokens

# 후처리: 치환된 복합어를 원래대로 복원
def restore_compounds(tokens):
    restored = []
    buffer = ""
    for token in tokens:
        if token == "_":
            continue
        buffer += token
        if buffer.replace("_", "") in compound_words:
            restored.append(buffer.replace("_", ""))  # 복합어 원래 형태로 복원
            buffer = ""
        else:
            restored.append(token)
            buffer = ""
    return restored

# 처리할 파일 리스트
review_files = [
    "review_data/beanplus_reviews.csv",
    "review_data/kinto_reviews.csv",
    "review_data/konu_reviews.csv",
    "review_data/locknlock_reviews.csv",
    "review_data/mosh_reviews.csv",
    "review_data/stanley_reviews.csv"
]

output_dir = "preprocessed_data"
os.makedirs(output_dir, exist_ok=True)

# 전처리 및 저장
tqdm.pandas()

for file in review_files:
    try:
        df = pd.read_csv(file, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(file, encoding='cp949')
        
    review_column = 'review'

    # 결측값 제거 및 중복 제거
    df = df.dropna(subset=[review_column])
    df = df.drop_duplicates(subset=[review_column])

    # 전처리 적용
    df['tokens'] = df[review_column].progress_apply(preprocess_review)

    # 후처리: 복합어 복원
    df['tokens'] = df['tokens'].apply(restore_compounds)

    # 파일 저장
    base_name = os.path.basename(file)
    output_file = os.path.join(output_dir, base_name.replace('.csv', '_preprocessed_okt.csv'))
    df.to_csv(output_file, encoding="utf-8-sig", index=False)
    print(f"Saved: {output_file}")